In [ ]:
import os
os.environ['CUDA_VISIBLE_DEVICES']='-1'
from pddlvm.utilities import tf, tfp, np, plt, tfb, floatf, tffloat, tfint, updateTfFloat,  _0

physical_devices = tf.config.list_physical_devices('GPU')
try:
  tf.config.set_logical_device_configuration(
    physical_devices[0],
    [tf.config.LogicalDeviceConfiguration(memory_limit=500),
     tf.config.LogicalDeviceConfiguration(memory_limit=500)])

  logical_devices = tf.config.list_logical_devices('GPU')
  assert len(logical_devices) == len(physical_devices) + 1
except:
  # Invalid device or cannot modify logical devices once initialized.
  pass
from pddlvm.poisson1DPDEState import PDE_State
from pddlvm.poisson1DSolver import Solver

tf.config.list_physical_devices()
print(floatf)

In [ ]:
#================= Test for PDEState =========================


omegaLength = 2.0
numFElems   = 400
xFE = tffloat( tf.linspace( -1., 1., numFElems + 1) )

pdeState = PDE_State(xFE, dimK=1, dimU=3, dimF=1)
print(pdeState.dimK, pdeState.dimU)

t = 10
N = tf.range(0, t, dtype=floatf)
def bind(n):
    return pdeState.trigCheb(xFE, n)
TMat = tf.vectorized_map( bind, N )
#print(TMat)
plt.plot(tf.transpose(tf.tile(xFE[None,:], [t, 1])), tf.transpose(TMat) )
plt.show()
#print(tf.tile(xFE[None,:], [t, 1]))

In [ ]:
numC = 10
meanCoeffs = tf.random.normal(stddev=0.1, shape=[numC], dtype=floatf)
mean, diagCov = pdeState.gaussChebyGaussCoeffs(meanCoeffs,
                                            tf.constant(0.1, shape=[numC],dtype=floatf), x=xFE)
plt.plot(xFE, mean)
std2 = 2*tf.sqrt(diagCov)
plt.fill_between(xFE, mean + std2, mean- std2, alpha=0.2)
plt.show()

meanT, varT = pdeState.unscentedGaussTransf(mean, diagCov, tfb.Softplus(), k=1)
plt.plot(xFE, meanT)
std2T = 2*tf.sqrt(varT)
plt.fill_between(xFE, meanT + std2T, meanT- std2T, alpha=0.2)
plt.show()

plt.plot(xFE, pdeState.kappa_fun(xFE))
plt.show()

t=10
TMat = pdeState.TOnMesh(t)
#print(TMat)
plt.plot(tf.transpose(tf.tile(xFE[None,:], [t, 1])), tf.transpose(TMat) )
plt.show()
plt.show()
coeffs = np.repeat(0.1, pdeState.dimU)
plt.plot(pdeState.u_funXFE(coeffs[:, None]))
plt.show()


func = lambda x:np.sin(10.*x)
x = tf.linspace(-1., 1., 10_00)
y = func(x)
order  = 100
coeffs = pdeState.getChebyCoeffs_fromPoints(x, y, order)

plt.plot(x, pdeState.genChebyFun(coeffs[:,None], x), label='approx')
plt.plot(x, y, label='true')
plt.legend()
plt.show()

pdeState = PDE_State(xFE, dimK=1, dimU=5, dimF=1)
x = pdeState.xFE
yFE = pdeState.xFE ** 2
a   = pdeState.meshToChebyU(yFE) 
print(a)
plt.plot(x, pdeState.chebyOnMesh(a), label='approx - integrated')
plt.plot(x, yFE, label='true')
plt.legend()
plt.show()

In [ ]:
#================= Test for Solver =========================

omegaLength = 2.0
numFElems   = 50
xFE = tf.constant( tf.linspace( 0., omegaLength, numFElems + 1) ) - 1.
print(xFE)

pdeState = PDE_State(xFE, dimK=10, dimU=15, dimF=5)
pdeState.kappaBij = tfb.Chain( bijectors=[ tfb.Shift(tffloat(1.)), tfb.Softplus()] )

pdeState.kappa   = tf.random.normal(shape=pdeState.kappa.shape)
pdeState.forcing = tf.random.normal(mean=5., stddev=2., shape=pdeState.forcing.shape)

kappaField = pdeState.kappa_funCXFE()
plt.plot(range(kappaField.shape[0]), kappaField)
plt.show()

forceField = pdeState.forcing_funXFE()
plt.plot(range(forceField.shape[0]), forceField)
plt.show()


#LINEAR TEST
print("TEST LINEAR")
solver = Solver( pdeState, xFE, isNonLinear=False )
print(pdeState.dimK, pdeState.dimU)

plt.plot(solver.xFE, solver.solve_tf_linear(), c = 'b', label='inversion solve linear')


u_sln = solver.solve_tf_nonLinear( )
print(solver.optim_results)
plt.plot(solver.xFE, u_sln,c='g', label='non-linear solve')
plt.legend()
plt.show()

print("TEST NON-LINEAR")
soft = tfb.Softplus()
solver = Solver( pdeState, xFE, isNonLinear=True )
print(pdeState.dimK, pdeState.dimU)
uLin = solver.solve_tf_linear()
plt.plot(solver.xFE,uLin , c = 'b', label='inversion solve linear')

A, f  = solver.assemble_tf( uLin )
precond = tf.linalg.inv(A)

u_sln = solver.solve_tf_nonLinear( u_0=u_sln, preconditioner=precond )
print(solver.optim_results)
plt.plot(solver.xFE, u_sln,c='g', label='non-linear solve')
plt.legend()
plt.show()

In [ ]:
omegaLength = 2.0
numFElems   = 50
xFE = tf.constant( tf.linspace( 0., omegaLength, numFElems + 1) ) - 1.
print(xFE)
pdeState = PDE_State(xFE, dimK=10, dimU=30, dimF=1)

pdeState.kappaBij = tfb.Chain( bijectors=[tfb.Shift(tffloat(1.)), tfb.Softplus()] )
#pdeState.kappaBij = tfb.Identity()
pdeState.kappa = tf.random.normal(shape=pdeState.kappa.shape)
kappaField = pdeState.kappa_funCXFE()
plt.plot(range(kappaField.shape[0]), kappaField)
plt.show()

#LINEAR TEST
print("TEST LINEAR")
solver = Solver( pdeState, xFE, isNonLinear=False )
u_sol_lin = solver.solve_tf_linear()
plt.plot(solver.xFE, u_sol_lin , c = 'b', label='inversion solve linear')

#NONLINEAR TEST

print("TEST NONLINEAR")
solver = Solver( pdeState, xFE, isNonLinear=True )
import time
start = time.time()
uc_sln = solver.solve_tf_FESpectral_bfgs( )
print("time bfgs spec = ", time.time() - start, 's')
print(solver.optim_results)
plt.plot(solver.xFE, pdeState.u_funXFE( uc_sln[:,None] ),c='g', label='spectral - LNONINEAR problem')
plt.legend()
plt.show()

import time
start = time.time()
uc_sln = solver.solve_tf_bfgs( )
print("time bfgs spec = ", time.time() - start, 's')
print(solver.optim_results)
plt.plot(solver.xFE,  uc_sln,c='g', label='FE - NONLINEAR problem')
plt.legend()
plt.show()

start = time.time()
uc_sln = solver.solve_tf_nonLinear_lsq( )
print("time solve_tf_nonLinear_lsq spec = ", time.time() - start, 's')
print(solver.optim_results)
plt.plot(solver.xFE,  uc_sln,c='g', label='FE -solve_tf_nonLinear_lsq NONLINEAR problem')
plt.legend()
plt.show()

#u_sol_lin = solver.solve_tf_linear( )
plt.plot(solver.xFE, u_sol_lin , c = 'b', label='inversion solve linear')
uc0 = pdeState.getChebyCoeffs_fromPoints(pdeState.xFE, u_sol_lin, 29)
start = time.time()
uc_sln = solver.solve_tf_nonLinear_lsq_spectral( u_c = None )
print("time lsq spec = ", time.time() - start, 's')
print(solver.optim_results)
plt.plot(solver.xFE, pdeState.u_funXFE( uc_sln[:,None] ),c='g', label='non-linear solve spec')
plt.legend()
plt.show()



plt.plot(solver.xFE, u_sol_lin, c = 'b', label='inversion solve linear')
u_sln = solver.solve_tf_nonLinear( )
print(solver.optim_results)
plt.plot(solver.xFE, u_sln,c='g', label='non-linear solve mesh')
plt.legend()
plt.show()

plt.plot(solver.xFE, pdeState.u_funXFE( uc_sln[:,None] ),c='g', label='non-linear solve spectral')
plt.plot(solver.xFE, u_sln,c='g', label='non-linear solve FE mesh')
plt.legend()
plt.show()



In [ ]:
import numpy as np


coeffs      = pdeState.getChebyCoeffs_fromPoints(pdeState.xFE, u_sol_lin, 19)
yFromCoeffs = pdeState.u_funXFE(coeffs[:,None])
plt.plot(pdeState.xFE, u_sol_lin, label='lin solve FE')
plt.plot(pdeState.xFE, yFromCoeffs, '--x', label='from np cheby')
plt.plot(pdeState.xFE, pdeState.u_funXFE(coeffs[:,None]), label='from u_funXFE FE')
plt.plot(solver.xFE, pdeState.u_funXFE( uc_sln[:,None] ), label='non-linear solve_tf_nonLinear_lsq_spectral')

#plt.plot(pdeState.xFE, pdeState.u_funXFE(solver.optim_results.x[:,None]), label='from u_funXFE FE--lsq coeffs')
plt.legend()

plt.show()
print(coeffs)
print(solver.optim_results.x)

#LINEAR TEST
print("TEST LINEAR")
solver = Solver( pdeState, xFE, isNonLinear=False )

u_sol_lin = solver.solve_tf_linear()
plt.plot(solver.xFE, u_sol_lin , c = 'b', label='inversion solve linear')

u_0  = tf.constant(0., shape=[pdeState.dimU], dtype=floatf)
uc_sln = solver.solve_tf_nonLinear_lsq_spectral( u_c = u_0)
print(solver.optim_results)
plt.plot(solver.xFE, pdeState.u_funXFE( uc_sln[:,None] ),c='g', label='non-linear solve lsq spec')
plt.legend()
plt.show()

plt.scatter(range(coeffs.shape[0]), tf.math.log(tf.math.abs(coeffs)), label='np coeffs')
plt.scatter(range(uc_sln.shape[0]), tf.math.log(tf.math.abs(uc_sln)), label='spectral sln ceffs')
plt.legend()
plt.show()

